In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

In [2]:
from keras.layers import Conv2D,Dense,Activation,MaxPooling2D,Flatten,BatchNormalization,Dropout,InputLayer
from keras.models import Sequential

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [4]:
class cnn_model:
    def __init__(self,input_shape):
        self.input_shape = input_shape
        self.model = Sequential()
        self.model.add(InputLayer(input_shape=input_shape))
        
    def add_cnn_block(self,nfilters=16,filter_size=(2,2),activation='relu',pool_size=(2,2),batch_norm='yes',dropout=None):
        self.model.add(Conv2D(nfilters,filter_size,padding="same"))
        if batch_norm == 'yes':
            self.model.add(BatchNormalization())
        self.model.add(Activation(activation))
        if dropout is not None:
            self.model.add(Dropout(dropout))
        self.model.add(MaxPooling2D(pool_size=pool_size,strides=(2,2),padding="same"))
        
    def add_dense_layer(self,neurons=16):
        self.model.add(Flatten())
        self.model.add(Dense(neurons,activation='relu'))
        
    def build_model(self,output,optimizer='adam',loss='categorical_crossentropy'):
        self.model.add(Dense(output,activation='softmax'))
        self.model.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])
        self.model.summary()
        return self.model

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

pref = '../inaturalist_12K/'
 
train_generator = train_datagen.flow_from_directory(
    pref+'train',
    target_size=(224,224),
    batch_size=16,
    class_mode='categorical')

val_generator = test_datagen.flow_from_directory(
    pref+'val',
    target_size=(224,224),
    batch_size=16,
    class_mode='categorical')

Found 8999 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [7]:
keras.backend.clear_session()
model = cnn_model((224,224,3))
model.add_cnn_block(nfilters=16,filter_size=(7,7),dropout=0.2)
model.add_cnn_block(nfilters=16,filter_size=(7,7),dropout=0.2)
model.add_cnn_block(nfilters=16,filter_size=(7,7),dropout=0.2)
model.add_dense_layer(16)
training = model.build_model(10)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 16)      2368      
_________________________________________________________________
batch_normalization (BatchNo (None, 224, 224, 16)      64        
_________________________________________________________________
activation (Activation)      (None, 224, 224, 16)      0         
_________________________________________________________________
dropout (Dropout)            (None, 224, 224, 16)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 16)      12560     
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 16)      6

In [9]:
training.fit_generator(train_generator,
                   steps_per_epoch=200,
                   epochs=10,
                   validation_data=val_generator,
                   validation_steps=50,
                   use_multiprocessing = True,
                   workers = 4)

Epoch 1/10
200/200 [==============================] - 24s 119ms/step - loss: 2.3030 - accuracy: 0.1025 - val_loss: 2.3032 - val_accuracy: 0.1375
Epoch 2/10
200/200 [==============================] - 25s 124ms/step - loss: 2.3034 - accuracy: 0.1097 - val_loss: 2.3024 - val_accuracy: 0.1050
Epoch 3/10
200/200 [==============================] - 31s 157ms/step - loss: 2.3038 - accuracy: 0.0831 - val_loss: 2.3012 - val_accuracy: 0.1225
Epoch 4/10
200/200 [==============================] - 28s 140ms/step - loss: 2.3033 - accuracy: 0.1013 - val_loss: 2.2999 - val_accuracy: 0.1350
Epoch 5/10
200/200 [==============================] - 29s 143ms/step - loss: 2.3014 - accuracy: 0.1119 - val_loss: 2.2999 - val_accuracy: 0.1300
Epoch 6/10
200/200 [==============================] - 29s 145ms/step - loss: 2.3034 - accuracy: 0.0975 - val_loss: 2.3013 - val_accuracy: 0.1350
Epoch 7/10
200/200 [==============================] - 31s 157ms/step - loss: 2.3045 - accuracy: 0.0988 - val_loss: 2.2998 - val_ac